## 关系拟合(回归 Regression)

In [82]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib qt5  # figure窗口显示动态图像

生成数据

In [83]:
# x = torch.linspace(-1, 1, 100)  # 注意别拼错linspace  # torch.Size([100])
# x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=0)  # torch.Size([1, 100])
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # torch.Size([100, 1])
print('x size:', x.size())  
y = x.pow(2) + 0.2*torch.rand(x.size())
print('y size:', y.size()) 
x, y = Variable(x), Variable(y)
# x, y = Variable(x, requires_grad=True), Variable(y, requires_grad=True)

x size: torch.Size([100, 1])
y size: torch.Size([100, 1])


打印散点图

In [84]:
plt.scatter(x.data.numpy(), y.data.numpy())
plt.show()

定义Net类

In [85]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):  # 定义层,但未搭建好神经网络
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)
        
    def forward(self, x):  # 搭建好神经网络
        x = F.relu(self.hidden(x))  # 是x不是self.x
        x = self.predict(x)  # 激活函数会有截断效果，所以末层一般不适用激活函数
        return x

搭建神经网络、优化器和损失函数

In [86]:
net = Net(1, 10, 1)  
print(net)  # 查看所搭建的网络
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
loss_func = torch.nn.MSELoss()

Net(
  (hidden): Linear(in_features=1, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=1, bias=True)
)


训练网络，绘制动态图像

In [89]:
plt.ion()  # 动态显示图像
# plt.show()
for t in range(500):
    # prediction = net.forward(x) # 自动调用forward 
    # model(x)的时候，底层自动调用forward方法计算结果
    prediction = net(x)  # torch.Size([100, 1])  # 注意可以不调用forward() 
    # print('Prediction size:', prediction.size())
    loss = loss_func(prediction, y)
    # 优化三步走
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 每5步打印一次
    if t % 5 == 0:
        plt.cla()  # 清楚当前figure中活动的axes
        plt.scatter(x.data.numpy(), y.data.numpy())  # 原图
        plt.plot(x.data.numpy(), prediction.data.numpy(), 'r-', lw=5)  # 拟合线段
        plt.text(0.5, 0, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size':10, 'color':'red'})
        plt.pause(0.1)
plt.ioff()
plt.show()